# Evaluation
This code contains all the necessary code to perform the necessary evaluations of the individual parts.

In [152]:
file_output_exercise1 = "../output.txt"
file_output_exercise2_rdd = "../output_rdd.txt"
file_output_exercise2_ds = "../output_ds.txt"

## Load results

In [153]:
def parse_file(file_path):
    category_terms = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()    

    merged_dict = None

    for line in lines:
        if line.startswith('<'):
            line_splitted = line.split(' ')
            category = line_splitted[0].replace('<', '').replace('>', '')
            terms = line_splitted[1:]
            for term_chisquared in terms:
                term, chi_squared = term_chisquared.split(':', 2)
                if category not in category_terms:
                    category_terms[category] = {}

                category_terms[category][term] = float(chi_squared)
        else:

            merged_dict = line.split(' ')
                
       
    return category_terms, merged_dict

In [154]:
chi_squared_ex1, merged_dict_ex1 = parse_file(file_output_exercise1)
chi_squared_ex2_rdd, merged_dict_ex2_rdd = parse_file(file_output_exercise2_rdd)
_, merged_dict_ex2_ds = parse_file(file_output_exercise2_ds)

## Part 1

In [155]:
from collections import defaultdict

def compareResults(chi_squared1, chi_squared2):
    result = {}
    missing_categories = defaultdict(list)
    for category in chi_squared1:
        result[category] = {}

        for term in chi_squared1[category]:
            result[category][term] = True
            if term in chi_squared2[category]:
                result[category][term] &= abs(chi_squared1[category][term] - chi_squared2[category][term]) < 0.001
            else:
                result[category][term] &= False
                missing_categories[category].append(((term, chi_squared1[category][term]), 'mapreduce'))

    for category in chi_squared2:
        for term in chi_squared2[category]:
            if term not in chi_squared1[category]:
                result[category][term] = False
                missing_categories[category].append(((term, chi_squared2[category][term]), 'rdd'))

    return result, missing_categories


In [156]:
result, missing_categories = compareResults(chi_squared_ex1, chi_squared_ex2_rdd)

In [157]:
def print_results(result):
    cnt = 0
    for category, term_difference in result.items():
        for term, is_different in term_difference.items():
            if is_different == False:
                cnt += 1
                print(f"Category: {category} Term: {term}")

    print(f"Total of different terms: {cnt}")

print_results(result)

Category: Apps_for_Android Term: devs
Category: Apps_for_Android Term: glu
Category: Apps_for_Android Term: teamlava
Category: Apps_for_Android Term: zynga
Category: Tools_and_Home_Improvement Term: bidet
Category: Tools_and_Home_Improvement Term: kidde
Total of different terms: 6


In [158]:
missing_categories

defaultdict(list,
            {'Apps_for_Android': [(('devs', 86.64962060177764), 'mapreduce'),
              (('glu', 86.64962060177764), 'mapreduce'),
              (('teamlava', 86.64962060177764), 'rdd'),
              (('zynga', 86.64962060177764), 'rdd')],
             'Tools_and_Home_Improvement': [(('bidet', 159.72357729489363),
               'mapreduce'),
              (('kidde', 159.72357729489363), 'rdd')]})

## Part 2

In [159]:
def containsAll(merged_dict_rdd, merged_dict_ds):
    not_contained = []
    for term in merged_dict_rdd:
        if term not in merged_dict_ds:
            not_contained.append(term)

    return not_contained

In [166]:
result = containsAll(merged_dict_ex2_rdd, merged_dict_ex2_ds)

In [168]:
len(merged_dict_ex2_rdd), len(merged_dict_ex2_ds), len(result)

(1464, 2000, 1250)

## Part 3